In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#export
from __future__ import annotations
from smpr3d.util import fourier_coordinates_2D, imsave, array_split_divpoints_ntotal
import matplotlib.pyplot as plt
from typing import TYPE_CHECKING, List, NamedTuple, Optional, Union, Callable
from smpr3d.util import fourier_coordinates_2D
import torch as th
import numpy as np 
from numpy.fft import fftshift
from dataclasses import dataclass
from torch.utils.data import BatchSampler, SequentialSampler
from smpr3d.util import advanced_raster_scan, ZernikeProbeSingle, beamlet_samples, natural_neighbor_weights
from numpy.fft import fftfreq
from smpr3d.algorithm import admm, ADMMOptions
from smpr3d.data import LinearIndexEncoded4DDataset, Dense4DDataset, Metadata4D

In [ ]:
#export 

@dataclass
class ReconstructionOptions:
    algorithm : str 
    algorithm_options : ADMMOptions
    compute_device : List[th.device]
    auxiliary_variable_device : th.device
    n_iter : int
    n_batches : int
    radius : float
    n_angular_samples : int
    n_radial_samples : int
    
    def __init__(self, algorithm = 'admm', 
                 algorithm_options = ADMMOptions(),
                 compute_device = [th.device(f'cuda:{i}') for i in [0]],
                 auxiliary_variable_device = th.device(f'cpu'),
                 n_iter = 20,
                 n_batches = 20,
                 radius = 15,
                 n_angular_samples = 6,
                 n_radial_samples = 4,):
        self.algorithm = algorithm
        self.algorithm_options = algorithm_options
        self.compute_device = compute_device
        self.auxiliary_variable_device = auxiliary_variable_device
        self.n_iter = n_iter
        self.n_batches = n_batches
        self.radius = radius
        self.n_angular_samples = n_angular_samples
        self.n_radial_samples = n_radial_samples




In [ ]:
#export 


from smpr3d.algorithm import SMPRSolution
from smpr3d.data import SMeta
def reconstruct_smatrix(data : Union[LinearIndexEncoded4DDataset, Dense4DDataset], 
                        vacuum_probe_dataset : Union[LinearIndexEncoded4DDataset, Dense4DDataset],
                        metadata : Metadata4D, 
                        options : ReconstructionOptions) -> SMPRSolution:
           
    r = advanced_raster_scan(data.scan_dimensions[0], 
                             data.scan_dimensions[1], 
                             fast_axis=1, 
                             mirror=[1, 1], 
                             theta=metadata.rotation_deg, 
                             dy=metadata.pixel_step[0],                         
                             dx=metadata.pixel_step[1])
    
    sampler = BatchSampler(SequentialSampler(range(len(vacuum_probe_dataset))), batch_size=len(vacuum_probe_dataset), drop_last=False)
    vacuum_probe_intensities = 0
    max_intensity = 0
    if isinstance(vacuum_probe_dataset, Dense4DDataset):
        for batch_inds in sampler:
            max_batch = th.max(th.sum(vacuum_probe_dataset[batch_inds], dim=(1,2)))
            if max_batch > max_intensity:
                max_intensity = max_batch.item()
            print(max_intensity)
            vacuum_probe_intensities += th.sum(vacuum_probe_dataset[batch_inds], dim=(0,))
    elif isinstance(vacuum_probe_dataset, LinearIndexEncoded4DDataset):
        for batch_inds in sampler:
            vacuum_batch = vacuum_probe_dataset[batch_inds]
            print(vacuum_batch)
            print(vacuum_batch.indices.dtype)
            print(type(vacuum_batch.indices))
            max_batch = th.max(th.sum(vacuum_batch.counts, dim=1))
            if max_batch > max_intensity:
                max_intensity = max_batch.item()
            w = th.zeros(tuple(vacuum_batch.frame_dimensions), dtype=th.float32)
            take = vacuum_batch.indices != th.iinfo(vacuum_batch.indices.dtype).max
            w.view(-1).scatter_add_(0, 
                                    th.tensor(vacuum_batch.indices[take]).long(), 
                                    th.tensor(vacuum_batch.counts[take], dtype=th.float32))
            vacuum_probe_intensities += w
    
    vacuum_probe0 = vacuum_probe_intensities / len(vacuum_probe_dataset)
    vacuum_probe = vacuum_probe0 * (vacuum_probe0 > vacuum_probe0.max() * 15e-2)
    vacuum_probe = vacuum_probe * (max_intensity / vacuum_probe.sum())
    vacuum_probe = vacuum_probe.cpu().numpy()
    
    if options.algorithm_options.verbose:
        fig, ax = plt.subplots()
        ax.imshow(fftshift(vacuum_probe))
        plt.title(f'Vacuum probe intensity: {max_intensity:2.2g} (should be centered)')
        plt.show()
    
    radius = options.radius
    n_angular_samples = options.n_angular_samples
    n_radial_samples = options.n_radial_samples
    
    take_beams = vacuum_probe > vacuum_probe.max() * 0.3
    dx = 1 / 2 / metadata.k_max
    print('data.frame_dimensions',data.frame_dimensions)
    print('dx',dx)
    
    MY, MX = data.frame_dimensions
    NY = int((np.ceil((r.max(axis=0)[0] + MY + options.algorithm_options.margin) / MY) * MY).item())
    NX = int((np.ceil((r.max(axis=0)[1] + MX + options.algorithm_options.margin) / MX) * MX).item())
    N = th.as_tensor([NY, NX])
    M = th.as_tensor([MY, MX])
        
    s_meta = SMeta(take_beams, take_beams, radius, dx, N.numpy(), M.numpy(), options.compute_device[0])
    s_meta = s_meta.make_beamlet_meta(n_radial_samples, n_angular_samples)
       
    if options.algorithm_options.verbose:
        fig, ax = plt.subplots(2, s_meta.natural_neighbor_weights.shape[1] // 2 + 1, figsize=(35, 10))
        axs = ax.ravel()
        for j in range(s_meta.natural_neighbor_weights.shape[1]):
            imax = axs[j].imshow(fftshift(s_meta.beamlets[j].cpu().numpy()))
            axs[j].set_xticks([])
            axs[j].set_yticks([])
        axs[-1].imshow(fftshift(vacuum_probe))
        plt.show()

    qnp = fourier_coordinates_2D(data.frame_dimensions, dx, centered=False)
    q = th.as_tensor(qnp)
    
    Psi_gen = ZernikeProbeSingle(q, metadata.wavelength, fft_shifted=True)
    Ap0 = th.as_tensor(vacuum_probe).float()
    C1 = th.as_tensor(metadata.aberrations)
    
    Psi_model = Psi_gen(C1, Ap0).to(options.compute_device[0]).detach()
    Psi_init = s_meta.beamlets * Psi_model[None, ...]
    Psi_init = Psi_init * (np.sqrt(vacuum_probe.sum())/th.norm(th.sum(Psi_init,0)))
    
    psi0 = th.fft.ifft2(Psi_init, norm='ortho')
    
    if options.algorithm_options.verbose:
        print(f"Psi_init norm: {th.norm(th.sum(Psi_init,0))**2}")
        print(f"psi_init norm: {th.norm(th.sum(psi0,0))**2}")
        
        fig, ax = plt.subplots(2, s_meta.Bp // 2 + 1, figsize=(35, 10))
        axs = ax.ravel()
        for j in range(s_meta.Bp):
            imax = axs[j].imshow(imsave(psi0[j].cpu().numpy()))
            axs[j].set_xticks([])
            axs[j].set_yticks([])
        axs[-1].imshow(fftshift(vacuum_probe))
        plt.show()
        
    if options.algorithm_options.verbose:
        print(f"B           : {s_meta.B}")
        print(f"Bp          : {s_meta.Bp}")
        print(f"K           : {len(data)}")
        print(f"NY, NX      : {NY},{NX}")
        print(f"MY, MX      : {MY},{MX}")
        
    if options.algorithm == 'admm':
        res = admm(data, r, psi0, s_meta, options.n_iter, options.n_batches, options.algorithm_options)
    else:
        res = admm(data, r, psi0, s_meta, options.n_iter, options.n_batches, options.algorithm_options)
        
    return res